In [1]:
import pandas as pd
import torch
from sklearn.utils import shuffle

In [2]:
import pandas as pd
rnames = ['userId', 'movieId', 'rating', "TimeStamp"]
ratings = pd.read_table("ml-1m.inter", header=0, names=rnames, engine='python')
ratings

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [3]:
ratings = shuffle(ratings)
ratio = 0.8
# ratings
train = ratings.copy()
test = ratings.copy()
train.iloc[int(ratio*len(ratings)):,2] = None
# train.iloc[int(ratio*len(ratings)):,'rating'] = 0
test.iloc[:int(ratio*len(ratings)),2] = None

In [6]:
ratings.describe()

,userId,movieId,rating,TimeStamp
count,1.000209e+06,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00,9.722437e+08
std,1.728413e+03,1.096041e+03,1.117102e+00,1.215256e+07
min,1.000000e+00,1.000000e+00,1.000000e+00,9.567039e+08
25%,1.506000e+03,1.030000e+03,3.000000e+00,9.653026e+08
50%,3.070000e+03,1.835000e+03,4.000000e+00,9.730180e+08
75%,4.476000e+03,2.770000e+03,4.000000e+00,9.752209e+08
max,6.040000e+03,3.952000e+03,5.000000e+00,1.046455e+09


In [7]:
rating_matrix = train.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = train['rating'].min(), train['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)
print(rating_matrix)
print(n_users*n_movies-rating_matrix.isnull().values.sum())

movieId  1     2     3     4     5     6     7     8     9     10    ...   
userId                                                               ...   
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...  \
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
6036      NaN   NaN   NaN  0.25   NaN   0.5   NaN   NaN   NaN   NaN  ...   
6037      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6038      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6039      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
6040      0.5   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

movieId  39

In [8]:
# Replacing missing ratings with -1 so we can filter them out later

rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)
print(rating_matrix)

tensor([[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        ...,
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
        [ 0.5000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000]])


In [9]:
class PMF(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, matrix, u_features, v_features):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        predicted_rating = torch.sigmoid(torch.mm(u_features, v_features.t()))
        
        return predicted_rating

In [10]:
class PMFLoss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
    
    def forward(self, matrix, predicted, u_features, v_features):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff*non_zero_mask)

        u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))
        
        # u_regularization = self.lam_u * torch.sum(u_features**2)
        # v_regularization = self.lam_v * torch.sum(v_features**2)   

        return prediction_error + u_regularization + v_regularization

In [11]:
# criterion = PMFLoss()
# loss = criterion(rating_matrix, user_features, movie_features)

In [12]:
# PMF Actual training loop now
pmf = PMF()
pmfloss = PMFLoss(lam_u=0.1, lam_v=0.1)
num_epoch = 100
latent_vectors = 20
user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
user_features.data.mul_(0.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
movie_features.data.mul_(0.01)

# pmferror = PMFLoss(lam_u=0.1, lam_v=0.1)
optimizer = torch.optim.Adam([user_features, movie_features], lr=0.01)
for step, epoch in enumerate(range(num_epoch)):
    optimizer.zero_grad()
    pred = pmf(rating_matrix, user_features, movie_features)
    loss = pmfloss(rating_matrix, pred, user_features, movie_features)
    loss.backward()
    optimizer.step()
    if step % 10 == 0:
        print(f"Step {step}, {loss:.3f}")

Step 0, 79315.461
Step 10, 73074.055
Step 20, 52552.934
Step 30, 43297.285
Step 40, 40154.367
Step 50, 38541.102
Step 60, 37294.383
Step 70, 36096.809
Step 80, 34826.090
Step 90, 33489.664


In [14]:
test_rating_matrix = test.pivot(index='userId', columns='movieId', values='rating')

test_rating_matrix[test_rating_matrix.isnull()] = -1
test_rating_matrix = torch.FloatTensor(test_rating_matrix.values)
test_rating_matrix

tensor([[ 5., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        ...,
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.],
        [-1., -1., -1.,  ..., -1., -1., -1.]])

In [15]:
## test General PMF
non_zero_mask = (test_rating_matrix != -1).type(torch.FloatTensor)
num = torch.sum(non_zero_mask)
# print(num)
# print(non_zero_mask)
predicted_ratings = torch.sigmoid(torch.mm(user_features, movie_features.t()))
# predicted_ratings = non_zero_mask * predicted_ratings
predicted_ratings = (predicted_ratings*(max_rating - min_rating) + min_rating)*non_zero_mask
actual_ratings = test_rating_matrix*non_zero_mask
# print(predicted_ratings)
# print(actual_ratings)

AE_diff = torch.abs(predicted_ratings - actual_ratings)
SE_diff = (predicted_ratings - actual_ratings)**2
# print(AE_diff)
# print(SE_diff)
test_MAE = torch.sum(AE_diff)/num
test_RMSE = torch.sqrt(torch.sum(SE_diff)/num)
# print(torch.count_nonzero(predicted_ratings).item())
# print(torch.count_nonzero(actual_ratings).item())
print('test_MAE =', test_MAE.data.numpy())
print('test_RMSE =', test_RMSE.data.numpy())

test_MAE = 0.69212526
test_RMSE = 0.8813581
